Load the drive in colab

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Load the images

In [ ]:
link='https://drive.google.com/open?id=1KIEFAfFqvmesQq8MJYjjj4hhqNVQH3--'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 

downloaded.GetContentFile('resized_images.zip')

1KIEFAfFqvmesQq8MJYjjj4hhqNVQH3--


In [ ]:
import os

path='resized_images.zip'
isExist = os.path.exists(path) 
print(isExist)

True


In [ ]:
!unzip resized_images.zip

Load annotations

In [ ]:
link='https://drive.google.com/open?id=1FXRe0ddA08afjRzAUu8WjjRuVvd5B3-j'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 

downloaded.GetContentFile('annotations.zip')
!unzip annotations.zip

1FXRe0ddA08afjRzAUu8WjjRuVvd5B3-j
Archive:  annotations.zip
   creating: annotations/
  inflating: __MACOSX/._annotations  
  inflating: annotations/test.json   
  inflating: __MACOSX/annotations/._test.json  
  inflating: annotations/train.json  
  inflating: __MACOSX/annotations/._train.json  
  inflating: annotations/val.json    
  inflating: __MACOSX/annotations/._val.json  


import libraries

In [ ]:
import sys
import numpy as np
import pandas as pd
from matplotlib.pyplot import imread
import pickle
from os import listdir
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
import time

import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model

from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform

from keras.engine.topology import Layer
from keras.regularizers import l2
from keras import backend as K

from sklearn.utils import shuffle

import numpy.random as rng

Using TensorFlow backend.


In [ ]:
import re
import numpy as np
from PIL import Image

from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.layers import Activation
from keras.layers import Input, Lambda, Dense, Dropout, Convolution2D, MaxPooling2D, Flatten
from keras.models import Sequential, Model
from keras.optimizers import RMSprop

In [ ]:

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

from keras.models import Model

In [ ]:
filename='resized_images/0_2.png'
image = load_img(filename, target_size=(299, 299))
image = img_to_array(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
image.shape

(1, 299, 299, 3)

Siamese Architecture

In [ ]:
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Input, Subtract, Lambda
from keras import optimizers
from keras.applications.inception_v3 import InceptionV3
def get_siamese_model(input_shape):
    
    
    # Define the tensors for the two input images
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    
    convolutional_net=InceptionV3()
    # Generate the encodings (feature vectors) for the two images
    encoded_image_1 = convolutional_net(left_input)
    encoded_image_2 = convolutional_net(right_input)

    # L1 distance layer between the two encoded outputs
    # One could use Subtract from Keras, but we want the absolute value
    l1_distance_layer = Lambda(
        lambda tensors: K.abs(tensors[0] - tensors[1]))
    l1_distance = l1_distance_layer([encoded_image_1, encoded_image_2])

    # Same class or not prediction
    prediction = Dense(units=1, activation='softmax')(l1_distance)
    
    # Connect the inputs with the outputs
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    optimizer = optimizers.Adam(lr = 10e-4)
            

    siamese_net.compile(loss="binary_crossentropy",optimizer=optimizer)
    
    # return the model
    return siamese_net

In [ ]:
model=get_siamese_model((299,299,3))
print(model.summary())

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
inception_v3 (Model)            (None, 1000)         23851784    input_10[0][0]                   
                                                                 input_11[0][0]                   
__________________________________________________________________________________________________
lambda_4 (Lambda)               (None, 1000)         0           inception_v3[1][0]         

Extract features from siamese model

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.applications.inception_v3 import preprocess_input
from pickle import dump
import os

def extract_features(directory):
  model=get_siamese_model((299,299,3))
  print(model.summary())
  features=dict()
  count=0
  res=[]
  # for name in listdir(directory):
  #   res.append(name)
  # res.sort()
  # print(res)
  # print("-----------------")
  for name in range(15000):
    # load an image from file
    filename = directory + '/' +str(name)+'.png'
    # temp=name.split('.')
    filename2=directory + '/' + str(name)+'_2.png'
    
    if os.path.exists(filename2)==False or os.path.exists(filename)==False:
      continue
    count+=1
    if count%100==0:
      print(count)
      # if count==100:
      #   return features
    # print(filename," ",filename2)
    image1 = load_img(filename2, target_size=(299, 299))
    # convert the image pixels to a numpy array
    image1 = img_to_array(image1)
    # reshape data for the model
    image1 = image1.reshape((1,image1.shape[0], image1.shape[1], image1.shape[2]))
    # print(image1.shape)
    # prepare the image for the InceptionV3 model
    image1 = preprocess_input(image1)

    image2 = load_img(filename, target_size=(299, 299))
    # convert the image pixels to a numpy array
    image2 = img_to_array(image2)
    # reshape data for the model
    image2 = image2.reshape((1,image2.shape[0], image2.shape[1], image2.shape[2]))
    
    image2 = preprocess_input(image2)
    # get features
    inputs=[image1,image2]
    feature = model.predict(inputs, verbose=0)
    # get image id
    # image_id = name.split('.')[0]
    # store feature
    features[name] = feature
    # print('>%s' % name)
  return features

# extract features from all images
directory = 'resized_images'
features = extract_features(directory)
print('Extracted Features: %d' % len(features))
# save to file
dump(features, open('features_sia_V3.pkl', 'wb'))
dump(features, open('drive/My Drive/171IT248/CV2/features_sia_V3.pkl', 'wb'))



Load the already dumped features if required

In [ ]:
link='https://drive.google.com/file/d/13QsGtwJp22Uw99Gx3ySCG0bxerhTCGBI/view?usp=sharing'
link='https://drive.google.com/open?id=13QsGtwJp22Uw99Gx3ySCG0bxerhTCGBI'

fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 

downloaded.GetContentFile('features_sia_V3.pkl')

13QsGtwJp22Uw99Gx3ySCG0bxerhTCGBI


In [ ]:
link='https://drive.google.com/open?id=1LzV8daMhM1Pw9tLkgX14hNtx6SpwfEBH'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 

downloaded.GetContentFile('model-ep001-loss2.800-val_loss2.660.h5')

Load the already dumped model if required

In [ ]:
link='https://drive.google.com/open?id=1maAvcRlgf9B2lXSquh5muyqa1MT-3yA4'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 

downloaded.GetContentFile('model_sia15.h5')

1maAvcRlgf9B2lXSquh5muyqa1MT-3yA4


In [ ]:
features[1],features[2],features[3],features[89],features[14957]

(array([[0.49958086]], dtype=float32),
 array([[0.49950066]], dtype=float32),
 array([[0.49948406]], dtype=float32),
 array([[0.5004282]], dtype=float32),
 array([[0.49964324]], dtype=float32))

Load the descriptions from annotations and cleaning it and save it

In [ ]:
import string
import json
# load doc into memory
def load_doc(filename):
    # open the file as read only
    mapping=dict()
    brr=json.loads(open('annotations/train.json').read())
    for i in range(len(brr)):
      image_id=brr[i]['img_id']
      curr=brr[i]['sentences']
      if image_id not in mapping:
        mapping[image_id] = list()

      for image_desc in curr:
        if image_desc not in mapping[image_id]:
          mapping[image_id].append(image_desc)
      
    crr=json.loads(open('annotations/test.json').read())
    for i in range(len(crr)):
      image_id=crr[i]['img_id']
      curr=crr[i]['sentences']
      if image_id not in mapping:
        mapping[image_id] = list()
      for image_desc in curr:
        if image_desc not in mapping[image_id]:
          mapping[image_id].append(image_desc)

    arr=json.loads(open('annotations/val.json').read())
    for i in range(len(arr)):
      image_id=arr[i]['img_id']
      curr=arr[i]['sentences']
      if image_id not in mapping:
        mapping[image_id] = list()
      for image_desc in curr:
        if image_desc not in mapping[image_id]:
          mapping[image_id].append(image_desc)
        
    return mapping

# extract descriptions for images
def load_descriptions(doc):
    mapping = dict()
    # process lines
    for line in doc.split('\n'):
        # split line by white space
        tokens = line.split()
        if len(line) < 2:
            continue
        # take the first token as the image id, the rest as the description
        image_id, image_desc = tokens[0], tokens[1:]
        # remove filename from image id
        image_id = image_id.split('.')[0]
        # convert description tokens back to string
        image_desc = ' '.join(image_desc)
        # create the list if needed
        if image_id not in mapping:
            mapping[image_id] = list()
        # store description
        mapping[image_id].append(image_desc)
    return mapping
 
def clean_descriptions(descriptions):
    # prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)
    for key, desc_list in descriptions.items():
        for i in range(len(desc_list)):
            desc = desc_list[i]
            # tokenize
            #print("desc",desc)
            desc = desc.split()
            # convert to lower case
            desc = [word.lower() for word in desc]
            # remove punctuation from each token
            desc = [w.translate(table) for w in desc]
            # remove hanging 's' and 'a'
            desc = [word for word in desc if len(word)>1]
            # remove tokens with numbers in them
            desc = [word for word in desc if word.isalpha()]
            # store as string
            desc_list[i] =  ' '.join(desc)
 
# convert the loaded descriptions into a vocabulary of words
def to_vocabulary(descriptions):
    # build a list of all description strings
    all_desc = set()
    for key in descriptions.keys():
        [all_desc.update(d.split()) for d in descriptions[key]]
    return all_desc
 
# save descriptions to file, one per line
def save_descriptions(descriptions, filename):
    lines = list()
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            lines.append(key + ' ' + desc)
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()
 
filename = 'annotations'
# load descriptions
descriptions = load_doc(filename)
# parse descriptions
# descriptions = load_descriptions(doc)
print(descriptions['1'])
print('Loaded: %d ' % len(descriptions))
# clean descriptions
clean_descriptions(descriptions)
# summarize vocabulary
vocabulary = to_vocabulary(descriptions)
print('Vocabulary Size: %d' % len(vocabulary))
# save to file
save_descriptions(descriptions, 'descriptions.txt')

['the third man in the group of three men is standing further away from the people he is with', 'two people standing next to the patch of grass are missing', 'two people walking toward the other are not there', 'the trunk of the car is open']
Loaded: 11329 
Vocabulary Size: 2246


Decoder using LSTM and using Tokenizer



In [ ]:
from numpy import array
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint
import json

# load doc into memory
def load_doc(filename):
	# open the file as read only
  file = open(filename, 'r')
  # read all text
  text = file.read()
  # close the file
  file.close()
  return text

# load a pre-defined list of photo identifiers
def load_set(filename):
  brr=json.loads(open(filename).read())
  dataset=[]
  for i in range(len(brr)):
    dataset.append(brr[i]['img_id'])

  return set(dataset)

# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
  # load document
  doc = load_doc(filename)
  descriptions = dict()
  for line in doc.split('\n'):
    # split line by white space
    # print("line:",line)
    tokens = line.split()
    # split id from description
    image_id, image_desc = tokens[0], tokens[1:]
    # skip images not in the set
    if image_id in dataset:
      # create list
      if image_id not in descriptions:
        descriptions[image_id] = list()
      # wrap description in tokens
      desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
      # store
      descriptions[image_id].append(desc)
  return descriptions

# load photo features
def load_photo_features(filename, dataset):
	# load all features
	all_features = load(open(filename, 'rb'))
	# filter features
	features = {k: all_features[int(k)] for k in dataset}
	return features

# covert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc

# fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
	lines = to_lines(descriptions)
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

# calculate the length of the description with the most words
def max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)

# create sequences of images, input sequences and output words for an image
def create_sequences(tokenizer, max_length, descriptions, photos, vocab_size):
	X1, X2, y = list(), list(), list()
	# walk through each image identifier
	for key, desc_list in descriptions.items():
		# walk through each description for the image
		for desc in desc_list:
			# encode the sequence
			seq = tokenizer.texts_to_sequences([desc])[0]
			# split one sequence into multiple X,y pairs
			for i in range(1, len(seq)):
				# split into input and output pair
				in_seq, out_seq = seq[:i], seq[i]
				# pad input sequence
				in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
				# encode output sequence
				out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
				# store
				X1.append(photos[key][0])
				X2.append(in_seq)
				y.append(out_seq)
	return array(X1), array(X2), array(y)
 
# define the model
def define_model(vocab_size, max_length):
	# feature extractor model
	inputs1 = Input(shape=(1,))
	fe1 = Dropout(0.5)(inputs1)
	fe2 = Dense(512, activation='relu')(fe1)
	# sequence model
	inputs2 = Input(shape=(max_length,))
	se1 = Embedding(vocab_size, 512, mask_zero=True)(inputs2)
	se2 = Dropout(0.5)(se1)
	se3 = LSTM(512)(se2)
	# decoder model
	decoder1 = add([fe2, se3])
	decoder2 = Dense(512, activation='relu')(decoder1)
	outputs = Dense(vocab_size, activation='softmax')(decoder2)
	# tie it together [image, seq] [word]
	model = Model(inputs=[inputs1, inputs2], outputs=outputs)
	model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
	# summarize model
	print(model.summary())
	plot_model(model, to_file='model.png', show_shapes=True)
	return model

In [ ]:
# load training dataset (6K)
filename = 'annotations/train.json'
train = load_set(filename)
print('Dataset: %d' % len(train))
# descriptions
train_descriptions = load_clean_descriptions('descriptions.txt', train)
print("ds",train_descriptions)
print('Descriptions: train=%d' % len(train_descriptions))
# photo features
train_features = load_photo_features('features_sia_V3.pkl', train)
print('Photos: train=%d' % len(train_features))
# prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# determine the maximum sequence length
max_length = max_length(train_descriptions)
print('Description Length: %d' % max_length)
# prepare sequences
X1train, X2train, ytrain = create_sequences(tokenizer, max_length, train_descriptions, train_features, vocab_size)

# dev dataset

# load test set
filename = 'annotations/val.json'
test = load_set(filename)
print('Dataset: %d' % len(test))
# descriptions
print("len:::",len(test))
test_descriptions = load_clean_descriptions('descriptions.txt', test)
print("ds",test_descriptions)
print('Descriptions: test=%d' % len(test_descriptions))
# photo features
test_features = load_photo_features('features_sia_V3.pkl', test)
print('Photos: test=%d' % len(test_features))
# prepare sequences
X1test, X2test, ytest = create_sequences(tokenizer, max_length, test_descriptions, test_features, vocab_size)

Dataset: 8566
ds {'76': ['startseq there are no differences endseq'], '235': ['startseq the person in the blue shirt from the image on the left is no longer present endseq', 'startseq the dark grey suv is gone in the photo on the right endseq'], '243': ['startseq the car in the upper left corner of the picture is gone endseq', 'startseq the two people in the upper part of the picture are gone endseq', 'startseq the person near ther handicapped parking place is not in the second picture endseq', 'startseq there appears to be figure maybe man that was near handicapped parking place stnaindg near pole in the bottom of the picture endseq', 'startseq the two people who were on th sidewalk are not in the second picture endseq', 'startseq there are two people in the seconde picture that are near the handicapped parking place endseq'], '103': ['startseq the blue car has moved endseq', 'startseq person appeared in the second picture endseq'], '32': ['startseq the people in the image are in diff

In [ ]:
X1train.shape,X2train.shape,ytrain.shape,X1test.shape,X2test.shape,ytest.shape

In [ ]:
X1train

array([[0.49979398],
       [0.49979398],
       [0.49979398],
       ...,
       [0.4988203 ],
       [0.4988203 ],
       [0.4988203 ]], dtype=float32)

Fit require high computing

In [ ]:
model = define_model(vocab_size, max_length)
# define checkpoint callback
filepath = 'model-ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
# fit model
model.fit([X1train, X2train], ytrain, epochs=1, verbose=True, callbacks=[checkpoint], validation_data=([X1test, X2test], ytest))


In [ ]:
from keras.models import load_model
model=load_model('model_sia15.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [ ]:
model = define_model(vocab_size, max_length)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 54)           0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 54, 512)      1030144     input_2[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 1)            0           input_1[0][0]                    
____________________________________________________________________________________________

Train With Progressive Loading

This section is for  not having enough memory to train the model  (e.g. cannot use AWS EC2 for whatever reason or High computing).

In [ ]:
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint

def create_sequences(tokenizer, max_length, desc_list, photo, vocab_size):
	X1, X2, y = list(), list(), list()
	# walk through each description for the image
	for desc in desc_list:
		# encode the sequence
		seq = tokenizer.texts_to_sequences([desc])[0]
		# split one sequence into multiple X,y pairs
		for i in range(1, len(seq)):
			# split into input and output pair
			in_seq, out_seq = seq[:i], seq[i]
			# pad input sequence
			in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
			# encode output sequence
			out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
			# store
			X1.append(photo)
			X2.append(in_seq)
			y.append(out_seq)
	return array(X1), array(X2), array(y)
# data generator, intended to be used in a call to model.fit_generator()
def data_generator(descriptions, photos, tokenizer, max_length, vocab_size):
	# loop for ever over images
	while 1:
		for key, desc_list in descriptions.items():
			# retrieve the photo feature
			photo = photos[key][0]
			in_img, in_seq, out_word = create_sequences(tokenizer, max_length, desc_list, photo, vocab_size)
			yield [[in_img, in_seq], out_word]
 
 
# define the model
# model = define_model(vocab_size, max_length)
# train the model, run epochs manually and save after each epoch
epochs = 15
steps = len(train_descriptions)
for i in range(22,epochs+20):
  # create the data generator
  generator = data_generator(train_descriptions, train_features, tokenizer, max_length, vocab_size)
  # fit for one epoch
  model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=True)
  # save model
  model.save('model_sia' + str(i) + '.h5')
  model.save('drive/My Drive/171IT248/CV2/model1/model_sia' + str(i) + '.h5')

Epoch 1/1
8566/8566 [==============================] - 1688s 197ms/step - loss: 1.9216 - accuracy: 0.4991
Epoch 1/1
8566/8566 [==============================] - 1892s 221ms/step - loss: 1.9312 - accuracy: 0.4978
Epoch 1/1
8566/8566 [==============================] - 1968s 230ms/step - loss: 1.9319 - accuracy: 0.4978
Epoch 1/1
8566/8566 [==============================] - 1897s 221ms/step - loss: 1.9437 - accuracy: 0.4966
Epoch 1/1
8566/8566 [==============================] - 1905s 222ms/step - loss: 1.9457 - accuracy: 0.4972
Epoch 1/1
5711/8566 [===================>..........] - ETA: 10:31 - loss: 1.9734 - accuracy: 0.4983

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip3 install "git+https://github.com/salaniz/pycocoevalcap.git"


  Cloning https://github.com/salaniz/pycocoevalcap.git to /tmp/pip-req-build-hpwgquyh
  Running command git clone -q https://github.com/salaniz/pycocoevalcap.git /tmp/pip-req-build-hpwgquyh
  Created wheel for pycocoevalcap: filename=pycocoevalcap-1.1-cp36-none-any.whl size=104310216 sha256=76672ea1dcb24e3323f4be9fd13e65358ac2bc3d2df7fc0563840cbf4932d74b
  Stored in directory: /tmp/pip-ephem-wheel-cache-yoi21p2m/wheels/43/c5/1e/8c540096ae74b7789563f1ba2a0e195308ee86f445a0cda110
Successfully built pycocoevalcap


Evaluate Model on Testing images



 

*   This involves passing in the start description token   ‘startseq‘, generating one word, then calling the model recursively with generated words as input until the end of sequence token is reached ‘endseq‘ or the maximum description length is reached.
*   The function below named generate_desc() implements this behavior and generates a textual description given a trained model, and a given prepared photo as input. It calls the function word_for_id() in order to map an integer prediction back to a word.

BLEU scores are used in text translation for evaluating translated text against one or more reference translations.

Here, we compare each generated description against all of the reference descriptions for the photograph. We then calculate BLEU scores for 1, 2, 3 and 4 cumulative n-grams.







In [ ]:
from numpy import argmax
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu


# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load a pre-defined list of photo identifiers
def load_set(filename):
  brr=json.loads(open(filename).read())
  dataset=[]
  for i in range(len(brr)):
    dataset.append(brr[i]['img_id'])

  return set(dataset)

# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# create list
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap description in tokens
			desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			# store
			descriptions[image_id].append(desc)
	return descriptions

# load photo features
def load_photo_features(filename, dataset):
	# load all features
	all_features = load(open(filename, 'rb'))
	# filter features
	features = {k: all_features[int(k)] for k in dataset}
	return features

# covert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc

# fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
	lines = to_lines(descriptions)
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

# calculate the length of the description with the most words
def max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)

# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
	# seed the generation process
	in_text = 'startseq'
	# iterate over the whole length of the sequence
	for i in range(max_length):
		# integer encode input sequence
		sequence = tokenizer.texts_to_sequences([in_text])[0]
		# pad input
		sequence = pad_sequences([sequence], maxlen=max_length)
		# predict next word
		yhat = model.predict([photo,sequence], verbose=0)
		# convert probability to integer
		yhat = argmax(yhat)
		# map integer to word
		word = word_for_id(yhat, tokenizer)
		# stop if we cannot map the word
		if word is None:
			break
		# append as input for generating the next word
		in_text += ' ' + word
		# stop if we predict the end of the sequence
		if word == 'endseq':
			break
	return in_text

# evaluate the skill of the model
def evaluate_model(model, descriptions, photos, tokenizer, max_length):
	actual, predicted = list(), list()
	# step over the whole set
	for key, desc_list in descriptions.items():
		# generate description
		yhat = generate_desc(model, tokenizer, photos[key], max_length)
		# store actual and predicted
		references = [d.split() for d in desc_list]
		actual.append(references)
		predicted.append(yhat.split())
	# calculate BLEU score
	print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
	print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
	print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
	print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))


# prepare tokenizer on train set

# load training dataset (6K)
filename = 'annotations/train.json'
train = load_set(filename)
print('Dataset: %d' % len(train))
# descriptions
train_descriptions = load_clean_descriptions('descriptions.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))
# prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# determine the maximum sequence length
max_length = max_length(train_descriptions)
print('Description Length: %d' % max_length)

# prepare test set

# load test set
filename = 'annotations/test.json'
test = load_set(filename)
print('Dataset: %d' % len(test))
# descriptions
test_descriptions = load_clean_descriptions('descriptions.txt', test)
print('Descriptions: test=%d' % len(test_descriptions))
# photo features
test_features = load_photo_features('features_sia_V3.pkl', test)
print('Photos: test=%d' % len(test_features))

# load the model
# filename = 'model_15.h5'
# model = load_model(filename)
# evaluate model
evaluate_model(model, test_descriptions, test_features, tokenizer, max_length)

Dataset: 8566
Descriptions: train=8566
Vocabulary Size: 2012
Description Length: 54
Dataset: 1270
Descriptions: test=1270
Photos: test=1270
BLEU-1: 0.440519
BLEU-2: 0.259316
BLEU-3: 0.191045
BLEU-4: 0.099098


Saving Tokenizer

In [ ]:
from keras.preprocessing.text import Tokenizer
from pickle import dump

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load a pre-defined list of photo identifiers
def load_set(filename):
  brr=json.loads(open(filename).read())
  dataset=[]
  for i in range(len(brr)):
    dataset.append(brr[i]['img_id'])

  return set(dataset)

# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# create list
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap description in tokens
			desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			# store
			descriptions[image_id].append(desc)
	return descriptions

# covert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc

# fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
  lines = to_lines(descriptions)
  print("lines:",len(lines))
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(lines)
  return tokenizer

# load training dataset (6K)
filename = 'annotations/train.json'
train = load_set(filename)
print('Dataset: %d' % len(train))
# descriptions
train_descriptions = load_clean_descriptions('descriptions.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))
# prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

Dataset: 8566
Descriptions: train=8566
lines: 17672


In [ ]:
link2='https://drive.google.com/open?id=15sH-2Jt11QQ0Fe7LOJVswD1YyOo7E_--'
fluff2, id2 = link2.split('=')
print (id2) # Verify that you have everything after '='
downloaded2 = drive.CreateFile({'id':id2}) 

downloaded2.GetContentFile('76_2.png')

15sH-2Jt11QQ0Fe7LOJVswD1YyOo7E_--


In [ ]:
link2='https://drive.google.com/open?id=1DiWO9-3X74RqRi1xXDnA18PA5rXjJzAM'
fluff2, id2 = link2.split('=')
print (id2) # Verify that you have everything after '='
downloaded2 = drive.CreateFile({'id':id2}) 

downloaded2.GetContentFile('76.png')

1DiWO9-3X74RqRi1xXDnA18PA5rXjJzAM


Feature extraction for Testing images

In [ ]:
from pickle import load
from numpy import argmax
from keras.preprocessing.sequence import pad_sequences

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

from keras.models import Model
from keras.models import load_model
from keras.preprocessing import image
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.applications.resnet50 import preprocess_input
from pickle import dump
import numpy as np
import os

# extract features from each photo in the directory
def extract_features(filename1,filename2,model):
	# load the model
  


  res=[]
  # for name in listdir(directory):
  #   res.append(name)
  # res.sort()
  # print(res)
  # print("-----------------")

  # load an image from file
  filename = filename1
  # temp=name.split('.')
  filename2=filename2



  # print(filename," ",filename2)
  image1 = load_img(filename2, target_size=(299, 299))
  # convert the image pixels to a numpy array
  image1 = img_to_array(image1)
  # reshape data for the model
  image1 = image1.reshape((1,image1.shape[0], image1.shape[1], image1.shape[2]))
  # print(image1.shape)
  # prepare the image for the VGG model
  image1 = preprocess_input(image1)

  image2 = load_img(filename, target_size=(299, 299))
  # convert the image pixels to a numpy array
  image2 = img_to_array(image2)
  # reshape data for the model
  image2 = image2.reshape((1,image2.shape[0], image2.shape[1], image2.shape[2]))

  image2 = preprocess_input(image2)
  # get features
  inputs=[image1,image2]
  feature = model.predict(inputs, verbose=True)

  return feature

# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
  # seed the generation process
  in_text = 'startseq'
  # iterate over the whole length of the sequence
  for i in range(max_length):
    # integer encode input sequence
    sequence = tokenizer.texts_to_sequences([in_text])[0]
    # pad input
    sequence = pad_sequences([sequence], maxlen=max_length)
    # predict next word
    # print(photo.shape," ",sequence.shape)
    # sequence=sequence.reshape(54,)
    yhat = model.predict([photo,sequence], verbose=True)
    # convert probability to integer
    yhat = argmax(yhat)
    # map integer to word
    word = word_for_id(yhat, tokenizer)
    # stop if we cannot map the word
    if word is None:
      break
    # append as input for generating the next word
    in_text += ' ' + word
    # stop if we predict the end of the sequence
    if word == 'endseq':
      break
  return in_text

# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))
# pre-define the max sequence length (from training)
max_length = 54
# load the model
model = load_model('model_sia15.h5')
# load and prepare the photograph
model1=get_siamese_model((299,299,3))
print(model1.summary())


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
inception_v3 (Model)            (None, 1000)         23851784    input_4[0][0]                    
                                                                 input_5[0][0]                    
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 1000)         0           inception_v3[1][0]         

In [ ]:
photo = extract_features('2631_2.png','2631.png',model1)
# generate description

description = generate_desc(model, tokenizer, photo, max_length)
print(description)

1/1 [==============================] - 0s 16ms/step
startseq there is person walking in the parking lot endseq


Overview of Descriptions.txt file

In [ ]:
doc = load_doc('descriptions.txt')
print(doc)